<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/RougeL_Labeling_of_Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##QA pairs generated using llama8b on xquad

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=9329e0050d8645bac7f1f516bd8a013d61f20d7b43d336beb567e33f4d5b21ab
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [13]:
import json
import pandas as pd

file_path = '/content/drive/My Drive/entailment_clusters_Llama3.1-xquadAll-LOG.json'

with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.DataFrame(data)
print(df.head())

                                                                                            0  \
question                                                     كم نقطة تخلى عنها دفاع البانثرز؟   
original_answer                                       {'text': ['308'], 'answer_start': [29]}   
clusters                                    [[6 نقطة], [5 لاعبين.\nQuestion: كم لاعب يلعب ...   
total_avg_neg_log_likelihoods_for_clusters  [20.8753662109375, 0.9677209854125977, 20.8181...   

                                                                                            1  \
question                                    كم عدد الاستحواذات التي قام بها جاريد ألين في ...   
original_answer                                      {'text': ['136'], 'answer_start': [557]}   
clusters                                    [[في مسيرته، قام جاريد ألين بست استحواذات., جا...   
total_avg_neg_log_likelihoods_for_clusters  [18.88265037536621, 19.25707244873047, 16.8100...   

                            

#Test RougeL

In [8]:
from huggingface_hub import login
login()

In [9]:
from transformers import AutoTokenizer
from rouge_score import rouge_scorer

# Load tokenizer
model_name = "core42/jais-13b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], tokenizer=tokenizer)

# Example strings
pred_str = 'السلام عليكم كيف حالك'
label_str = 'السلام عليكم صديقي كيف حالك'

scores = scorer.score(label_str, pred_str)

for key in scores:
    print(f'{key}: {scores[key]}')

tokenizer_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

rougeL: Score(precision=1.0, recall=0.8, fmeasure=0.888888888888889)


#Use RougeL on data

In [25]:
import json
from rouge_score import rouge_scorer

# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Load data from JSON file
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

# Save labeled data to a new JSON file
def save_labels(output_data, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as file:
        json.dump(output_data, file, ensure_ascii=False, indent=4)

# Process hallucination detection
def check_hallucination(data):
    updated_data = {}

    for question_id, item in data.items():
        question = item['question']
        original_answer = item['original_answer']['text'][0]  # Ground truth
        non_hallucinated_count = 0
        updated_clusters = []

        # Process each cluster
        for cluster in item['clusters']:
            updated_cluster = []

            for answer in cluster:
                if answer.strip():  # If the answer is not empty, compute ROUGE
                    scores = scorer.score(original_answer, answer)
                    rouge_l_f1 = scores['rougeL'].fmeasure
                else:  # If the answer is empty, assign a score of 0.0 and label as Hallucinated
                    rouge_l_f1 = 0.0

                # Determine hallucination label
                label = "Non-Hallucinated" if rouge_l_f1 > 0.3 else "Hallucinated"

                # Count non-hallucinated answers
                if label == "Non-Hallucinated":
                    non_hallucinated_count += 1

                # Store answer with its score and label
                updated_cluster.append({
                    "text": answer,
                    "f1_score": round(rouge_l_f1, 2),
                    "label": label
                })

            # Append the processed cluster
            updated_clusters.append(updated_cluster)

        # Determine question-level label
        question_label = "Non-Hallucinated" if non_hallucinated_count >= 6 else "Hallucinated"

        # Store updated question with labeled clusters
        updated_data[question_id] = {
            "question": question,
            "original_answer": item["original_answer"],
            "clusters": updated_clusters,
            "total_avg_neg_log_likelihoods_for_clusters": item["total_avg_neg_log_likelihoods_for_clusters"],
            "question_label": question_label  # Add question-level label
        }

    return updated_data

# File paths
input_file_path = "/content/drive/My Drive/entailment_clusters_Llama3.1-xquadAll-LOG.json"
output_file_path = "labeled_data_rougel.json"

# Load data
data = load_data(input_file_path)

# Process and modify question labels
updated_data = check_hallucination(data)

# Save updated data
save_labels(updated_data, output_file_path)

print(f"Labeled data has been saved to {output_file_path}")


Labeled data has been saved to labeled_data_rougel.json
